In [1]:
#https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#dataframe-apis

In [2]:
#/home/hduser/Descargas/aoe_archive

matches:

* token: ID da partida
* winning_team: que equipo gañou a partida
* mirror: se os dous xogadores utilizaron a mesma civilizacion no xogo
* ladder: que tipo de partida clasificatoria era
* patch: en que parche do xogo foi a partida
* average rating: puntuacion media dos xogadores da partida
* map: que mapa do xogo se utilizou na partida
* map_size: tamaño dese mapa na partida
* num_players: cantos xogadores habia nesa partida
* server: en que servidor se librou a partida
* duration: duracion da partida

match_players:

* token: ID do xogador
* match: partida na que xogou (o ID de match)
* rating: calificacion do xogador nese momento
* color: que color de xogador tiña na partida
* civ: que civilizacion utilizou o xogador na partida
* team: en que equipo estaba o xogador
* winner: se o xogador gañou a partida ou non

1. Cantos rexistros contén o csv de matches.
2. Cantas veces se utilizou cada mapa.
3. Media dos ratings dos xogadores no dataset.
4. Partidas no mapa máis xogado para o servidor “westeurope”.
5. Máximo de duracion dunha partida en cada parche.
6. Mínimo de duracion dunha partida en cada tipo de clasificatoria (ladder).
7. Que parches diferentes estar rexistrados no dataset.
8. Suma dos xogadores de cada partida no dataset.
9. Canto se xogou cada cor (enriquece a táboa con match_players).
10. Civilizacions na partida “axps4XstiBOmrDeG“ (enriquece a táboa con match_players).

In [68]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Execicio Pighive") \
    .getOrCreate()

In [69]:
spark

In [77]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType

In [91]:
columnas = StructType([
    StructField("matchid",StringType(),True),
    StructField("wteam",IntegerType(),True),
    StructField("mirror",StringType(),True),
    StructField("ladder", StringType(),True),
    StructField("patch", DoubleType(),True),
    StructField("avgrating", DoubleType(),True),
    StructField("maptype", StringType(),True),
    StructField("mapsize", StringType(),True),
    StructField("numplayers", IntegerType(),True),
    StructField("server", StringType(),True),
    StructField("duration", StringType(),True)
    ])

In [133]:
partidas = spark.read.format("csv").load(path="matches.csv",schema=columnas,sep=',')

In [120]:
partidas

DataFrame[matchid: string, wteam: int, mirror: string, ladder: string, patch: double, avgrating: double, maptype: string, mapsize: string, numplayers: int, server: string, duration: string]

In [94]:
partidas.show(5)

+----------------+-----+------+-------+-------+---------+----------+--------+----------+-----------+--------+
|         matchid|wteam|mirror| ladder|  patch|avgrating|   maptype| mapsize|numplayers|     server|duration|
+----------------+-----+------+-------+-------+---------+----------+--------+----------+-----------+--------+
|           token| null|mirror| ladder|   null|     null|       map|map_size|      null|     server|duration|
|B2c2Ad6Uf6TVL5uk|    2| False| RM_1v1|39515.0|   1575.0|    arabia|    tiny|         2|brazilsouth| 0:23:30|
|eb8WdcbYlpeFLnoc|    2| False|RM_TEAM|39515.0|   2187.0|    arabia|   large|         8|     eastus| 0:32:48|
|tphUBvZ8eZBmai0L|    2| False|RM_TEAM|39515.0|   2450.0|megarandom|   large|         8|brazilsouth| 0:33:08|
|Kmhn0HZ8o8WwdXV9|    1| False|RM_TEAM|39515.0|   2387.0|    arabia|   large|         8|     ukwest| 0:25:51|
+----------------+-----+------+-------+-------+---------+----------+--------+----------+-----------+--------+
only showi

---

1. Cantos rexistros contén o csv de matches.

In [95]:
partidas.count()

3153768

2. Cantas veces se utilizou cada mapa.

In [96]:
partidas.groupBy('maptype').count().sort('count', ascending=False).show(10)

+------------+-------+
|     maptype|  count|
+------------+-------+
|      arabia|1174357|
|       arena| 422091|
|  four_lakes| 153459|
|     hideout| 152758|
|  golden_pit| 129842|
|  megarandom| 121077|
|golden_swamp|  81188|
|   hill_fort|  74651|
|       nomad|  69519|
|   lombardia|  68708|
+------------+-------+
only showing top 10 rows



3. Media dos ratings dos xogadores no dataset.

In [101]:
partidas.groupBy().avg().select('avg(avgrating)').show()

+------------------+
|    avg(avgrating)|
+------------------+
|1271.1566946746302|
+------------------+



4. Partidas no mapa máis xogado para o servidor “westeurope”.

In [109]:
partidas.groupBy('server').count().filter(partidas['server'] == 'westeurope').show()

+----------+-----+
|    server|count|
+----------+-----+
|westeurope|68841|
+----------+-----+



5. Máximo de duracion dunha partida en cada parche.

In [114]:
partidas.groupBy('patch').min('avgrating').show()

+-------+--------------+
|  patch|min(avgrating)|
+-------+--------------+
|41855.0|         101.0|
|39515.0|         144.0|
|   null|          null|
|43210.0|         202.0|
|37650.0|         721.0|
|42848.0|        1055.0|
|39284.0|         975.0|
|40220.0|         172.0|
|37906.0|         147.0|
|35584.0|         330.0|
|36202.0|         301.0|
|40874.0|         141.0|
|36906.0|         281.0|
+-------+--------------+



6. Mínimo de duracion dunha partida en cada tipo de clasificatoria (ladder).

In [115]:
partidas.groupBy('ladder').max('avgrating').show()

+-------+--------------+
| ladder|max(avgrating)|
+-------+--------------+
| ladder|          null|
|RM_TEAM|        3131.0|
| RM_1v1|        2504.0|
+-------+--------------+



7. Que parches diferentes estar rexistrados no dataset.

In [123]:
partidas.select('patch').distinct().show()

+-------+
|  patch|
+-------+
|41855.0|
|39515.0|
|   null|
|43210.0|
|37650.0|
|42848.0|
|39284.0|
|40220.0|
|37906.0|
|35584.0|
|36202.0|
|40874.0|
|36906.0|
+-------+



8. Suma dos xogadores de cada partida no dataset.

In [129]:
partidas.groupBy().sum('numplayers').show()

+---------------+
|sum(numplayers)|
+---------------+
|        9732500|
+---------------+



---

In [130]:
columnas2 = StructType([
    StructField("xogadorid",StringType(),True),
    StructField("matchid",StringType(),True),
    StructField("rating",DoubleType(),True),
    StructField("color", StringType(),True),
    StructField("civil", StringType(),True),
    StructField("team", IntegerType(),True),
    StructField("winner", StringType(),True)
    ])

In [134]:
xogadores = spark.read.format("csv").load(path="match_players.csv",schema=columnas2,sep=',')

In [135]:
xogadores

DataFrame[xogadorid: string, matchid: string, rating: double, color: string, civil: string, team: int, winner: string]

In [138]:
partidas.join(xogadores,['matchid','matchid'])

DataFrame[matchid: string, matchid: string, wteam: int, mirror: string, ladder: string, patch: double, avgrating: double, maptype: string, mapsize: string, numplayers: int, server: string, duration: string, xogadorid: string, rating: double, color: string, civil: string, team: int, winner: string]

---

9. Canto se xogou cada cor (enriquece a táboa con match_players).

In [143]:
partidas.join(xogadores,['matchid','matchid']).groupBy('color').count().show()

2022-02-15 18:15:29,994 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2022-02-15 18:15:30,592 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2022-02-15 18:15:30,914 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2022-02-15 18:15:31,436 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2022-02-15 18:15:33,119 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
2022-02-15 18:15:33,126 WARN expressions.RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+------+-------+
| color|  count|
+------+-------+
|Orange| 433475|
|  Cyan| 679950|
|  null|      1|
| Green|1103141|
|Purple| 682216|
|  Blue|2713926|
|Yellow|1036954|
|   Red|2714025|
|  Grey| 368812|
+------+-------+



10. Civilizacions na partida “axps4XstiBOmrDeG“ (enriquece a táboa con match_players).

In [141]:
partidas.join(xogadores,['matchid','matchid']).filter(partidas['matchid'] == 'axps4XstiBOmrDeG').select('civil').show()

+-------+
|  civil|
+-------+
|Chinese|
|  Goths|
+-------+

